In [1]:
from ultralytics.models.sam import Predictor as SAMPredictor
from ultralytics import SAM
import cv2
import json
import os
from typing import List, Tuple
import torch
import numpy as np
from custom_utils import load_images_and_boxes, plot_masks_on_images, save_masks, resize_images_and_masks

In [2]:
# 이미지와 레이블 경로
image_folder = 'data/images/'
label_folder = 'data/labels/'
mask_output_folder = 'data/masks/'  # 마스크를 저장할 폴더 경로

cfg = dict(conf=0.75, task="segment", model='weights/sam_l.pt', imgsz = 512,
                 project="./attempt")
print(cfg)

{'conf': 0.75, 'task': 'segment', 'model': 'weights/sam_l.pt', 'imgsz': 512, 'project': './attempt'}


In [7]:
# SAM 모델을 사용하여 다수의 이미지에 대해 마스크를 생성하는 함수
def generate_masks_with_sam(cfg, images: List[np.ndarray], all_boxes: List[List[List[float]]]) -> List[np.ndarray]:
    # SAM 모델 초기화
    sam = SAMPredictor(overrides=cfg)
    print(sam)
    # 결과를 저장할 리스트
    all_masks = []

    for image, boxes in zip(images, all_boxes):
        # 이미지를 PyTorch 텐서로 변환
        image_tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float()  # [1, C, H, W] 형식으로 변환

        # 바운딩 박스가 numpy 배열인지 확인 및 변환
        boxes_array = np.array(boxes, dtype=np.float32)
        # results = sam(image, bboxes=boxes)
        
        # masks = []
        # for r in results:
        #     if r.masks is not None and r.masks.data is not None:
        #         masks_data = r.masks.data.cpu().numpy()  # 텐서를 numpy 배열로 변환
        #         masks.append(masks_data)
        # all_masks.append(masks)
        sam.set_image(image_tensor)
        # SAM 모델을 사용하여 마스크 생성
        
        masks, scores, logits = sam.inference(bboxes=boxes)

        #생성된 마스크를 결과 리스트에 추가
        all_masks.append(masks)

        # 결과 출력 (선택 사항)
        print(f'scores: {scores}', f'logits: {logits}')

    return all_masks

In [6]:

# 이미지와 바운딩 박스 로드
images, all_boxes, image_names = load_images_and_boxes(image_folder, label_folder, max_images=5)

# 마스크 생성
masks = generate_masks_with_sam(cfg, images, all_boxes)

# 마스크와 이미지 리사이즈 (목표 크기: 512x384)
resized_images, resized_masks = resize_images_and_masks(images, masks, target_size=(384, 512))

 # 마스크 저장
save_masks(resized_masks, image_names, mask_output_folder)

# 결과 시각화
plot_masks_on_images(resized_images, resized_masks, image_names)

Ultralytics YOLOv8.2.83 🚀 Python-3.10.14 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)


TypeError: Predictor.inference() missing 1 required positional argument: 'im'